# Example FlowSOM Pipeline


This vignette describes a protocol for analyzing high-dimensional cytometry
data using FlowSOM, a clustering and visualization algorithm based
on a self-organizing map (SOM). FlowSOM is used to distinguish cell populations
from cytometry data in an unsupervised way and can help to gain deeper insights
in fields such as immunology and oncology.


## Loading in the data

FlowSOM handles different inputs, such as an anndata object by pytometry or a filepath. For this purpose we will make use of an anndata object. This allows
easier preprocessing.


In [63]:
# Import modules
import flowsom as fs
import pytometry as pm
import scanpy as sc
import csv
import numpy as np
import matplotlib.pyplot as plt

In [64]:
# Load data
ff = fs.io.read_FCS("../../tests/data/not_preprocessed.fcs")
ff

We can get an overview of the most important data in our anndata object by using `var`. All the metadata is stored in a dictionary at `ff.uns["meta]`


In [65]:
ff.var

In [66]:
ff.uns["meta"].keys()

Additionaly we can read in a csv file as well.


In [67]:
ff_csv = fs.io.read_csv("../../tests/data/fcs.csv")
ff_csv

The `FlowSOM` function accepts an anndata object or a filepath to a fcs or csv file. The `FlowSOM` function will return a `FlowSOM` mudata object. This object contains all the information about the SOM and the clustering.


In [68]:
# Visualize data
sc.pl.scatter(ff, x="CD4", y="CD8", color="CD8", size=2)

## Preprocessing

We start with compensating the data and then we transform
the data with the logicle function. For CyTOF data an arcsinh transformation is
preferred which is also found in the pytometry package. Besides
compensation and transformation, we also recommend cleaning the data by removing
margin events and by using cleaning algorithms.


In [69]:
# Compensate
ff_comp = pm.pp.compensate(ff, inplace=False)

# Transform
cols_to_use = [8, 11, 13, 14, 15, 16, 17]
ff_t = fs.pp.normalize_estimate_logicle(ff_comp, cols_to_use)

In [70]:
# Visualize data
ax = plt.hist2d(ff_t[:, "CD4"].X.flatten(), ff_t[:, "CD8"].X.flatten(), bins=200, cmin=1, cmap="jet")
plt.xlabel("CD4")
plt.ylabel("CD8")
plt.show()

# FlowSOM

The easiest way to use this package is using the wrapper function
FlowSOM, although it has less options than using the underlying functions separately. It holds the data in a MuData object, of which the first
modality is the cell data and the second modality is the cluster data. We will
cluster the data with a 10 x 10 SOM grid and 10 metaclusters. Notice that due to the
just-in-time compilation of numba, the first run of FlowSOM can take a while and
the subsequent runs will be much faster. We also set the seed here to make sure the analysis is deterministic and reproducible.

In [71]:
fsom = fs.FlowSOM(ff_t.copy(), cols_to_use=cols_to_use, n_clusters=10, xdim=10, ydim=10, seed=42)

We can inspect the underlying model and can see it's like any other scikit-learn model. The FlowSOM estimator first overclusters using a `cluster_model` (Self-Organizing Map). Then it uses a `metacluster_model` (Consensus Agglomerative Clustering) to merge the clusters into metaclusters.

In [72]:
fsom.model

The output is stored in a MuData object, containing two AnnData object: `cell_data` (n_cells x n_features) and `cluster_data` (n_SOM_nodes x n_features).

In [73]:
fsom.mudata

We can access the cell data and the cluster data with the `get_cell_data()` and
`get_cluster_data()` functions.

The _cell data_ is an anndata object that contains
the original cell data. As observations, we find the clustering, metaclustering
and distance to best matching unit per cell. In var, we find the pretty colnames,
i.e. a combination of markers and channels, the markers, the channels and a
boolean mask of the columns used for clustering. n_nodes and n_metaclusters in
uns contain the number of clusters and metaclusters respectively.


In [74]:
fsom.get_cell_data()

The _cluster data_ contains the original median values per cluster per marker.
In obsm, we find the cv values, sd values, mad values, coordinates of the nodes,
coordinates of the the grid and the coordinates of the MST layout.
The xdim, ydim, outliers, igraph object and metacluster MFIs can be found in uns.


In [75]:
fsom.get_cluster_data()

In [76]:
fsom.get_cell_data().var

## flowsom_clustering

Alternatively to working with this FlowSOM object, we can simply add the FlowSOM clustering and metaclustering to an existing AnnData object. The convenience function `flowsom_clustering` is available and is similar to other clustering methods in the scverse. The FlowSOM
clustering and metaclustering can be found in `.obs` and the parameters used in the
FlowSOM clustering in `.uns.FlowSOM`.

In [97]:
ff_clustered = fs.flowsom_clustering(ff_t, cols_to_use, xdim=10, ydim=10, n_clusters=10, seed=42)
ff_clustered

## FlowSOM visualizations

A FlowSOM object can be visualized with the `plot_stars()` function


In [78]:
p = fs.pl.plot_stars(fsom, background_values=fsom.get_cluster_data().obs.metaclustering)

We can also visualize the grid, to reduce overlap and get a better view of the data.
The node sizes of the nodes and/or the background nodes can be made equal with the equal_node_size or equal_background_size argument


In [79]:
p = fs.pl.plot_stars(
    fsom,
    background_values=fsom.get_cluster_data().obs.metaclustering,
    view="grid",
    equal_node_size=True,
    equal_background_size=True,
)

If you have a label for every cell, we can visualize this with `plot_pies()`.
Here every node is a piechart with the percentage of cells in each cluster.


In [80]:
# Read in that data
file = open("../../tests/data/gating_result.csv")
data = csv.reader(file)
data = [i[0] for i in data]

# Plot
p = fs.pl.plot_pies(fsom, data, background_values=fsom.get_cluster_data().obs.metaclustering)

We can also visualize the cluster numbers or metacluster numbers with the help of
`plot_numbers()`, if `level="clusters"` or `level="metaclusters"`, respectively.
This functions uses `plot_labels()` internally, to which one can pass custom labels,
such as the cell type labels.


In [81]:
p = fs.pl.plot_numbers(fsom, level="clusters", text_size=5)

It is possible to visualize one marker on a FlowSOM tree with the `plot_marker()`
function. This function uses the `plot_variable()` function internally, to which
one can pass custom variables, such as the cell type labels.


In [82]:
p = fs.pl.plot_marker(fsom, marker=np.array(["CD3"]))

We can also visualize the clusters and metaclusters on a 2D scatter plot with
`plot_2D_scatters()`.


In [83]:
p = fs.pl.plot_2D_scatters(
    fsom,
    channelpairs=[["CD3", "CD4"], ["CD19", "TCRb"]],
    clusters=[[1, 2], [3]],
    metaclusters=[[4], [5, 6]],
    density=True,
    centers=True,
)

# Downstream analysis

We might need the percentages, counts or the percentages
of positive cells per cluster or metacluster and per file for further analysis.
This can be done with the `get_features()` function. This function returns a
dictionary containing pandas of the requested data.


In [84]:
features = fs.tl.get_features(
    fsom,
    files=[ff_t[1:1000, :], ff_t[1000:2000, :]],
    level=["clusters", "metaclusters"],
    type=["counts", "percentages"],
)
features["metacluster_percentages"]

The counts, percentages, percentages_positive, per cluster or metacluser can also
extracted from a FlowSOM object. For this we can use `get_counts()`, `get_percentages()`,
`get_cluster_percentages_positive()` or `get_metacluster_percentages_positive`, respectively.


In [85]:
fs.tl.get_counts(fsom, level="clusters")

At last it is also possible to visualize a UMAP colored by the metaclustering or the expression of a marker. For this we will use `scanpy`.


In [86]:
# Get subset of the cell data
ref_markers_bool = fsom.get_cell_data().var["cols_used"]
subset_fsom = fsom.get_cell_data()[
    np.random.choice(range(fsom.get_cell_data().shape[0]), 5000, replace=False),
    fsom.get_cell_data().var_names[ref_markers_bool],
]
sc.pp.neighbors(subset_fsom)
sc.tl.umap(subset_fsom)

In [87]:
# By metaclustering
subset_fsom.obs["metaclustering"] = subset_fsom.obs["metaclustering"].astype(str)
sc.pl.umap(subset_fsom, color="metaclustering")

In [88]:
# By markers
sc.pl.umap(subset_fsom, color=fsom.get_cell_data().var_names[ref_markers_bool])

# Other interesting functions

To get the markers or channels from the corresponding channels or markers of an FCS or a FlowSOM object, we can use `get_markers()` or `get_channels()`.

newdata
subset


In [89]:
fs.tl.get_channels(ff_t, np.array(["CD3", "CD4"]))

In [90]:
fs.tl.get_markers(fsom, np.array(["PE-A", "PE-Cy7-A"]))

We can also merge multiple FCS files with random subsampling
with the function `aggregate_flowframes()`.


In [91]:
fs.pp.aggregate_flowframes(
    filenames=[
        "../../tests/data/not_preprocessed.fcs",
        "../../tests/data/not_preprocessed.fcs",
    ],
    c_total=5000,
)

To map new data on an existing FlowSOM object, we can use the `new_data()` function.


In [92]:
fsom_new = fsom.new_data(ff_t[1:200, :])
p = fs.pl.plot_stars(fsom_new, background_values=fsom_new.get_cluster_data().obs.metaclustering)

We can also take a subset of a FlowSOM object. For this we use the `subset()` function.


In [95]:
fsom_subset = fsom.subset(fsom.get_cell_data().obs["metaclustering"] == 2)
p = fs.pl.plot_stars(fsom_subset, background_values=fsom_subset.get_cluster_data().obs.metaclustering)